# CAI - Colony Avalanche Ecosystem Index

Crypto index tracking is a popular passive investment strategy. Since an index cannot be traded directly, index tracking refers to the process of creating a set of crypto assets managed by smart contract that approximates its performance. A straightforward way to do that is to purchase all the assets that compose an index in appropriate quantities.

In order to compensate for the price changes of the individual assets in the index we need to rebalance the portfolio. Choosing the optimal frequency is not an easy task and many factors (e.g., transaction costs, volatility, type of assets, etc.) need to be taken into account.

Conventional approaches to index portfolio rebalancing include periodic and tolerance band rebalancing. With periodic rebalancing, the portfolio is adjusted from its current weights back to the target weights at a consistent time interval (e.g., monthly or quarterly).

## Setting

In [1]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint

In [2]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [3]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots 

In [4]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion cretira

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data at least 6 months prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Avalanche. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.

1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have at least $2mm of onchain liquidity on a single pair.
    1. The token must have shown consistent DeFi liquidity on Avalanche.

1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available.



### Tokens list

In [5]:
cg.ping()

{'gecko_says': '(V3) To the Moon!'}

In [6]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True, order='market_cap_desc'))


In [7]:
tokens = {}
tokens_by_addr = {}
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'avalanche' == list(coin['platforms'].keys())[0]:
        symbol = ''.join(c for c in coin['symbol'] if c.isalnum()).upper()
        address = coin['platforms']['avalanche'].lower()
        token = {
            'symbol': symbol,
            'address': address 
        }
        tokens[symbol] = token
        tokens_by_addr[address] = token
        

In [8]:
tokens['USDC'], tokens['WAVAX']

({'symbol': 'USDC', 'address': '0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664'},
 {'symbol': 'WAVAX', 'address': '0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7'})

#### Market data

In [9]:
tokens_data = [] 
tokens_data_page = pd.DataFrame(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc'))
tokens_data.append(tokens_data_page)
for i in range(2, 20):
    tokens_data_page = pd.DataFrame(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))
    tokens_data.append(tokens_data_page)
tokens_data = pd.concat(tokens_data)
tokens_data = tokens_data.set_index(tokens_data['symbol'].str.upper())

In [10]:
tokens_data

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
BTC,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,20620.000000,394477142773,1,4.342889e+11,2.532868e+10,20844.000000,...,2.100000e+07,2.100000e+07,69045.000000,-70.11683,2021-11-10T14:24:11.849Z,67.810000,30327.76443,2013-07-06T00:00:00.000Z,None,2022-06-23T12:12:21.624Z
ETH,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1109.430000,134735176440,2,NaN,1.455862e+10,1119.510000,...,NaN,NaN,4878.260000,-77.33093,2021-11-10T14:24:19.604Z,0.432979,255306.48895,2015-10-20T00:00:00.000Z,"{'times': 70.91047221294849, 'currency': 'btc'...",2022-06-23T12:12:47.530Z
USDT,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.998410,67029267278,3,NaN,4.140683e+10,1.006000,...,6.697973e+10,NaN,1.320000,-24.45072,2018-07-24T00:00:00.000Z,0.572521,74.59429,2015-03-02T00:00:00.000Z,None,2022-06-23T12:11:11.803Z
USDC,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.000000,55851236036,4,NaN,4.807895e+09,1.008000,...,5.578290e+10,NaN,1.170000,-14.72620,2019-05-08T00:40:28.300Z,0.891848,12.12798,2021-05-19T13:14:05.611Z,None,2022-06-23T12:11:35.182Z
BNB,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,225.410000,36875380942,5,3.729089e+10,7.065920e+08,226.460000,...,1.632770e+08,1.651168e+08,686.310000,-67.18122,2021-05-10T07:24:17.097Z,0.039818,565571.72293,2017-10-19T00:00:00.000Z,None,2022-06-23T12:11:32.360Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GOGO,gogocoin,gogo,GOGOcoin,https://assets.coingecko.com/coins/images/2134...,0.079160,999834,1897,7.901438e+06,4.424600e+04,0.080641,...,9.999088e+07,1.000000e+08,0.829272,-90.43415,2021-12-08T19:38:29.521Z,0.078451,1.11583,2022-06-23T11:22:08.157Z,None,2022-06-23T12:13:10.663Z
FILDA,filda,filda,Filda,https://assets.coingecko.com/coins/images/1393...,0.005397,998042,1898,2.698624e+06,1.830091e+04,0.005481,...,5.000000e+08,5.000000e+08,2.360000,-99.77159,2021-02-20T01:00:26.617Z,0.004662,15.80738,2022-06-18T20:56:54.866Z,None,2022-06-23T12:12:36.774Z
PLR,pillar,plr,Pillar,https://assets.coingecko.com/coins/images/809/...,0.003848,997356,1903,NaN,8.547470e+03,0.003932,...,8.000000e+08,NaN,1.560000,-99.75395,2018-02-01T00:05:32.250Z,0.003124,22.80641,2022-06-18T20:54:21.898Z,None,2022-06-23T12:12:45.764Z


### Liquidity check

In [11]:
dex_subgraph_urls = {
    'Trader Joe': 'https://api.thegraph.com/subgraphs/name/traderjoe-xyz/exchange',
    'Pangolin': 'https://api.thegraph.com/subgraphs/name/dasconnor/pangolin-dex'
} 

In [12]:
query = """
    query pairs ($token0_ids: [ID!]!, $token1_ids: [ID!]!) {
        pairs (first:1000, where: {token0_in: $token0_ids, token1_in: $token1_ids}){
            id
            token0 {
              id
            }
            token1 {
              id
            }
            reserveUSD
        }
    }
    """

In [13]:
base_tokens = [tokens['WAVAX'], tokens['USDC']] 
base_tokens

[{'symbol': 'WAVAX', 'address': '0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7'},
 {'symbol': 'USDC', 'address': '0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664'}]

In [14]:
topn = 10000 
topn_tokens = [tokens[symb] 
               for symb in tokens_data[:topn].index 
               if symb in tokens.keys()]
topn_tokens.extend(base_tokens)

In [15]:
base_tokens_ids = [token['address'] for token in base_tokens]
tokens_ids = [token['address'] for token in topn_tokens] 

In [16]:
rows = {token['symbol']: {} for token in topn_tokens}
for dex_name, url in dex_subgraph_urls.items():
    request = requests.post(url, json={'query': query, 'variables': {'token0_ids': base_tokens_ids, 'token1_ids': tokens_ids}})
    pairs1 = request.json()['data']['pairs']

    request = requests.post(url, json={'query': query, 'variables': {'token0_ids': tokens_ids, 'token1_ids': base_tokens_ids}})
    pairs2 = request.json()['data']['pairs']
    
    pairs = pairs1 + pairs2
    
    print(f"{dex_name}: {len(pairs)} pairs")
    
    for pair in pairs:
        if pair['token0']['id'] in base_tokens_ids:
            base_token = tokens_by_addr[pair['token0']['id']]
            token = tokens_by_addr[pair['token1']['id']]
        else:
            base_token = tokens_by_addr[pair['token1']['id']]
            token = tokens_by_addr[pair['token0']['id']]
        
        colname = f"{dex_name}_{base_token['symbol']}" 
        
        rows[token['symbol']][colname] = float(pair['reserveUSD'])

Trader Joe: 69 pairs
Pangolin: 40 pairs


In [17]:
liquidities = pd.DataFrame.from_dict(rows, orient='index')
liquidities = liquidities.rename(index={'WAVAX': 'AVAX'})
liq_check = (liquidities > 2e6).any(axis=1).sort_values(ascending=False)
liq_check_passed = liq_check[liq_check]

In [18]:
liquidities

,Trader Joe_USDC,Trader Joe_WAVAX,Pangolin_USDC,Pangolin_WAVAX
USDT,7.778544e+05,6.888269e+06,5.785505e+04,4.100901e+06
SUSD,1.014665e+04,NaN,9.990253e-07,7.392432e-02
JOE,6.256203e+05,8.721569e+06,3.471811e+00,2.437509e+04
SAVAX,1.381404e-01,3.670940e+06,NaN,4.769991e+05
FITFI,3.041657e+00,8.219465e+03,NaN,3.930467e+04
SB,1.510778e-03,7.808595e+03,NaN,4.017539e+01
TIME,2.291381e-02,9.099100e+04,NaN,1.124715e+04
QI,2.208191e-04,8.668184e+04,NaN,2.531020e+06
THOR,9.640612e+00,2.650020e+06,NaN,8.316116e+00
XAVA,4.859454e-01,5.386660e+05,3.252086e+00,1.684345e+06


In [19]:
lq_passed = tokens_data[
    tokens_data.index.isin(liq_check_passed.index)
]

In [20]:
lq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
USDT,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.998410,67029267278,3,NaN,4.140683e+10,1.006000,...,6.697973e+10,NaN,1.32000,-24.45072,2018-07-24T00:00:00.000Z,0.572521,74.59429,2015-03-02T00:00:00.000Z,None,2022-06-23T12:11:11.803Z
AVAX,avalanche-2,avax,Avalanche,https://assets.coingecko.com/coins/images/1255...,17.790000,5017822800,17,1.284429e+10,4.833253e+08,18.020000,...,4.053718e+08,7.200000e+08,144.96000,-87.74589,2021-11-21T14:18:56.538Z,2.800000,534.17714,2020-12-31T13:15:21.540Z,None,2022-06-23T12:12:07.938Z
JOE,joe,joe,JOE,https://assets.coingecko.com/coins/images/1756...,0.229809,64473792,328,1.150568e+08,2.770467e+06,0.232847,...,3.776649e+08,5.000000e+08,5.09000,-95.46787,2021-11-21T14:27:00.202Z,0.026588,767.13151,2021-08-11T14:01:35.223Z,None,2022-06-23T12:12:38.134Z
SAVAX,benqi-liquid-staked-avax,savax,BENQI Liquid Staked AVAX,https://assets.coingecko.com/coins/images/2365...,18.110000,54823691,371,NaN,7.062530e+05,18.380000,...,3.021726e+06,NaN,103.55000,-82.49964,2022-04-02T13:44:54.612Z,14.030000,29.11494,2022-06-19T05:30:05.232Z,None,2022-06-23T12:12:44.839Z
QI,benqi,qi,BENQI,https://assets.coingecko.com/coins/images/1636...,0.012355,28921282,532,8.895462e+07,3.101348e+06,0.012376,...,7.200000e+09,7.200000e+09,0.39417,-96.86430,2021-08-24T03:58:11.390Z,0.008900,38.87504,2022-05-12T07:26:36.407Z,None,2022-06-23T12:12:37.996Z
QI,qi-dao,qi,Qi Dao,https://assets.coingecko.com/coins/images/1532...,0.321339,24701321,576,6.435512e+07,3.190550e+05,0.325807,...,2.000000e+08,2.000000e+08,6.09000,-94.73061,2021-06-16T09:41:28.898Z,0.093443,243.66239,2022-06-15T06:43:06.382Z,None,2022-06-23T12:12:44.852Z
THOR,thorswap,thor,THORSwap,https://assets.coingecko.com/coins/images/1929...,0.272085,22378443,592,1.360097e+08,1.109960e+05,0.274477,...,5.000000e+08,5.000000e+08,3.05000,-91.09859,2021-11-06T09:21:45.214Z,0.203189,33.71576,2022-06-18T21:08:00.820Z,None,2022-06-23T12:12:58.136Z


### Non quantative checks

In [21]:
exclude_list = [
    {
        'symbol': 'USDT',
        'id': 'tether',
        'reason': 'Stable coin'
    },
    {
        'symbol': 'QI',
        'id': 'qi-dao',
        'reason': 'Duplicate'
    },
    {
        'symbol': 'THOR',
        'id': 'thorswap',
        'reason': 'not Avalanche native'
    }
]
exclude_ids = [token['id'] for token in exclude_list]

In [22]:
nq_passed = lq_passed[
    ~lq_passed['id'].isin(exclude_ids)
]

In [23]:
nq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
AVAX,avalanche-2,avax,Avalanche,https://assets.coingecko.com/coins/images/1255...,17.790000,5017822800,17,1.284429e+10,483325336.0,18.020000,...,4.053718e+08,7.200000e+08,144.96000,-87.74589,2021-11-21T14:18:56.538Z,2.800000,534.17714,2020-12-31T13:15:21.540Z,None,2022-06-23T12:12:07.938Z
JOE,joe,joe,JOE,https://assets.coingecko.com/coins/images/1756...,0.229809,64473792,328,1.150568e+08,2770467.0,0.232847,...,3.776649e+08,5.000000e+08,5.09000,-95.46787,2021-11-21T14:27:00.202Z,0.026588,767.13151,2021-08-11T14:01:35.223Z,None,2022-06-23T12:12:38.134Z
SAVAX,benqi-liquid-staked-avax,savax,BENQI Liquid Staked AVAX,https://assets.coingecko.com/coins/images/2365...,18.110000,54823691,371,NaN,706253.0,18.380000,...,3.021726e+06,NaN,103.55000,-82.49964,2022-04-02T13:44:54.612Z,14.030000,29.11494,2022-06-19T05:30:05.232Z,None,2022-06-23T12:12:44.839Z
QI,benqi,qi,BENQI,https://assets.coingecko.com/coins/images/1636...,0.012355,28921282,532,8.895462e+07,3101348.0,0.012376,...,7.200000e+09,7.200000e+09,0.39417,-96.86430,2021-08-24T03:58:11.390Z,0.008900,38.87504,2022-05-12T07:26:36.407Z,None,2022-06-23T12:12:37.996Z


### Retrieving marketcaps and prices

In [24]:
min_lifetime = 180
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for index, data in nq_passed[['id', 'symbol']].iterrows():
    id_ = data['id']
    symbol = data['symbol'].upper()
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', symbol])
    df_prices = df_prices[df_prices[symbol] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < min_lifetime:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < {min_lifetime} days')
        exclude_list.append(symbol)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', symbol])
    df_mcaps = df_mcaps[df_mcaps[symbol] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < min_lifetime:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < {min_lifetime} days')
        
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)

Excluding SAVAX, prices data available only for 125 < 180 days


In [25]:
lt_passed = nq_passed[
    ~nq_passed.index.isin(exclude_list)
]

In [26]:
lt_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
AVAX,avalanche-2,avax,Avalanche,https://assets.coingecko.com/coins/images/1255...,17.790000,5017822800,17,1.284429e+10,483325336.0,18.020000,...,4.053718e+08,7.200000e+08,144.96000,-87.74589,2021-11-21T14:18:56.538Z,2.800000,534.17714,2020-12-31T13:15:21.540Z,None,2022-06-23T12:12:07.938Z
JOE,joe,joe,JOE,https://assets.coingecko.com/coins/images/1756...,0.229809,64473792,328,1.150568e+08,2770467.0,0.232847,...,3.776649e+08,5.000000e+08,5.09000,-95.46787,2021-11-21T14:27:00.202Z,0.026588,767.13151,2021-08-11T14:01:35.223Z,None,2022-06-23T12:12:38.134Z
QI,benqi,qi,BENQI,https://assets.coingecko.com/coins/images/1636...,0.012355,28921282,532,8.895462e+07,3101348.0,0.012376,...,7.200000e+09,7.200000e+09,0.39417,-96.86430,2021-08-24T03:58:11.390Z,0.008900,38.87504,2022-05-12T07:26:36.407Z,None,2022-06-23T12:12:37.996Z


### Marketcap ranking & top10 filtering

In [27]:
chosen_tokens = lt_passed.sort_values("market_cap", ascending=False).iloc[:10]

In [28]:
chosen_tokens

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
AVAX,avalanche-2,avax,Avalanche,https://assets.coingecko.com/coins/images/1255...,17.790000,5017822800,17,1.284429e+10,483325336.0,18.020000,...,4.053718e+08,7.200000e+08,144.96000,-87.74589,2021-11-21T14:18:56.538Z,2.800000,534.17714,2020-12-31T13:15:21.540Z,None,2022-06-23T12:12:07.938Z
JOE,joe,joe,JOE,https://assets.coingecko.com/coins/images/1756...,0.229809,64473792,328,1.150568e+08,2770467.0,0.232847,...,3.776649e+08,5.000000e+08,5.09000,-95.46787,2021-11-21T14:27:00.202Z,0.026588,767.13151,2021-08-11T14:01:35.223Z,None,2022-06-23T12:12:38.134Z
QI,benqi,qi,BENQI,https://assets.coingecko.com/coins/images/1636...,0.012355,28921282,532,8.895462e+07,3101348.0,0.012376,...,7.200000e+09,7.200000e+09,0.39417,-96.86430,2021-08-24T03:58:11.390Z,0.008900,38.87504,2022-05-12T07:26:36.407Z,None,2022-06-23T12:12:37.996Z


In [29]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [30]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

In [31]:
prices_data

,AVAX,JOE,QI
date,,,
2021-08-26,47.184934,1.269184,0.270208
2021-08-27,42.329113,1.863219,0.239757
2021-08-28,49.984485,2.272851,0.261765
2021-08-29,50.257387,2.023451,0.280079
2021-08-30,47.543874,2.531329,0.214812
...,...,...,...
2022-06-19,14.897482,0.191939,0.010790
2022-06-20,16.232279,0.211545,0.011815
2022-06-21,17.165201,0.225874,0.012031


In [32]:
marketcaps

,AVAX,JOE,QI
date,,,
2021-08-26,8.215166e+09,8.181409e+07,8.914565e+07
2021-08-27,7.408226e+09,1.257190e+08,7.817968e+07
2021-08-28,8.709313e+09,1.459951e+08,8.838145e+07
2021-08-29,8.769450e+09,1.361720e+08,9.130421e+07
2021-08-30,8.299403e+09,1.720847e+08,7.043803e+07
...,...,...,...
2022-06-19,4.175621e+09,5.355007e+07,2.400001e+07
2022-06-20,4.561591e+09,5.892893e+07,2.739022e+07
2022-06-21,4.832367e+09,6.293300e+07,2.809323e+07


### Assets data

In [33]:
tickers = list(prices_data.columns)

In [34]:
prices_data / prices_data.iloc[0] - 1

,AVAX,JOE,QI
date,,,
2021-08-26,0.000000,0.000000,0.000000
2021-08-27,-0.102910,0.468045,-0.112695
2021-08-28,0.059331,0.790796,-0.031244
2021-08-29,0.065115,0.594292,0.036534
2021-08-30,0.007607,0.994453,-0.205012
...,...,...,...
2022-06-19,-0.684275,-0.848770,-0.960067
2022-06-20,-0.655986,-0.833322,-0.956275
2022-06-21,-0.636214,-0.822032,-0.955474


In [35]:
fig = px.line(prices_data.pct_change(),
       labels={'variable': 'asset', 'value': 'return, %'})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    template='plotly_white',
    title='Assets daily returns'
)
fig.show()

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 50%.
* All excess weight is proportionally redistributed to all uncapped tokens.
* Any asset with a weight below 0.5% will be removed from the index.

### Iterative proportional adjusting

In [36]:
def adjust_weights(w, b):
    assert abs(w.sum() - 1) < 0.01, w.sum()
    assert b * w.shape[0] >=1
    wa = w.copy()
    
    while (wa > b).any():
        c = (wa[wa>b]-b).sum()
        wa[wa > b] = b
        w_less = wa[wa < b]
        wa[wa < b] += c * (w_less / w_less.sum())
    
    return wa

### Weight mcaps

In [37]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [38]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

In [39]:
weights

,AVAX,JOE,QI
date,,,
2021-08-26,0.979614,0.009756,0.010630
2021-08-27,0.973214,0.016516,0.010270
2021-08-28,0.973794,0.016324,0.009882
2021-08-29,0.974716,0.015135,0.010148
2021-08-30,0.971608,0.020146,0.008246
...,...,...,...
2022-06-19,0.981767,0.012591,0.005643
2022-06-20,0.981428,0.012679,0.005893
2022-06-21,0.981511,0.012782,0.005706


### Adjust weights

In [40]:
adjusted_weights = weights.apply(adjust_weights, args=[0.5], axis=1)
adjusted_weights = adjusted_weights.loc[:, adjusted_weights.iloc[-1] > 0.005]

In [41]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
    range=[0, 1],
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

In [42]:
adjusted_weights

,AVAX,JOE,QI
date,,,
2021-08-26,0.5,0.239279,0.260721
2021-08-27,0.5,0.308288,0.191712
2021-08-28,0.5,0.311454,0.188546
2021-08-29,0.5,0.299310,0.200690
2021-08-30,0.5,0.354781,0.145219
...,...,...,...
2022-06-19,0.5,0.345261,0.154739
2022-06-20,0.5,0.341343,0.158657
2022-06-21,0.5,0.345686,0.154314


### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [43]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)
    
    remainders = w_scaled - w_res
    
    k = round(remainders.sum())
    k_largest_rems = np.argsort(remainders)[::-1][:k]
    
    w_res[k_largest_rems] += 1
    
    return w_res 

In [44]:
last_weights = adjusted_weights.iloc[-1]
last_weights

AVAX    0.500000
JOE     0.343378
QI      0.156622
Name: 2022-06-23 00:00:00, dtype: float64

In [45]:
converted_last_weights = convert_weights(last_weights)
converted_last_weights

AVAX    127
JOE      88
QI       40
Name: 2022-06-23 00:00:00, dtype: int64

## Summary table

In [46]:
cai = pd.DataFrame()
cai.index = chosen_tokens.index
cai['name'] = chosen_tokens['name']
cai['market_cap'] = marketcaps.iloc[-1].astype(int)
cai['price'] = prices_data.iloc[-1]
cai['weight'] = last_weights
cai['weight_converted'] = converted_last_weights
cai['address'] = [tokens[symb]['address'] for symb in cai.index]

In [47]:
cai

,name,market_cap,price,weight,weight_converted,address
symbol,,,,,,
AVAX,Avalanche,4546159657,16.122561,0.500000,127,fvweahmxkfeig8snevq42hc6whryy3efyavebmqdndgcgxn5z
JOE,JOE,60353244,0.215403,0.343378,88,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
QI,BENQI,27528501,0.011737,0.156622,40,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5
